# 📘 Архитектуры Агентов 19: LATS + MCTS (Language Agent Tree Search)

В этом блокноте мы реализуем **Language Agent Tree Search (LATS)** — одну из самых продвинутых архитектур для задач сложного рассуждения. Она объединяет возможности языковых моделей с мощным алгоритмом поиска **Monte Carlo Tree Search (MCTS)**.

В то время как Tree of Thoughts (ToT) просто исследует варианты, LATS добавляет возможность **обучения на ошибках** в рамках одной задачи. Используя механизм самокритики (reflection) и обратного распространения оценки (backpropagation), агент может понимать, какие ветви рассуждений ведут к тупику, и фокусировать усилия на перспективных направлениях.

### Определение
**LATS (Language Agent Tree Search)** — это архитектура, в которой LLM используется как агент, рассуждающий в пространстве поиска, а MCTS управляет этим поиском. Она включает четыре ключевых этапа:
1. **Selection (Выбор):** Выбор наиболее перспективного узла дерева на основе UCB1 (Upper Confidence Bound).
2. **Expansion (Расширение):** Генерация новых шагов (мыслей) из выбранного узла.
3. **Evaluation/Reflection (Оценка):** Оценка качества шага с помощью LLM (самокритика).
4. **Backpropagation (Обратное распространение):** Обновление ценности всех родительских узлов на основе полученной оценки.

### Применение: Игра 24 (Game of 24)
Для демонстрации мы используем классическую математическую задачу "Игра 24". Даны 4 числа (например, `4, 9, 10, 13`), нужно использовать арифметические операции `(+, -, *, /)`, чтобы получить ровно 24. Это отличный бенчмарк для проверки способностей к планированию и перебору вариантов.

## Фаза 0: Основа и Настройка

### Шаг 0.1: Установка библиотек

**Что мы будем делать:**
Установим необходимые библиотеки. Нам понадобятся стандартные инструменты LangChain и LangGraph.

In [ ]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

### Шаг 0.2: Настройка окружения

**Что мы будем делать:**
Настроим доступ к API Nebius и инициализируем модель. Для этой задачи лучше использовать температуру > 0 для генерации разнообразных вариантов решений (мыслей).

In [ ]:
import os
import math
import random
import re
from typing import List, Dict, Any, Optional, Tuple
from collections import deque
from dotenv import load_dotenv

# LangChain & Graph
from langchain_nebius import ChatNebius
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

# UI
from rich.console import Console
from rich.tree import Tree
from rich.panel import Panel

from notebook_utils import setup_environment

console = setup_environment(
    project_name="Agentic Architecture - LATS (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

# Температура повыше для креативности в вариантах решений
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0.7)

### Шаг 0.3: Среда "Игра 24"

**Что мы будем делать:**
Реализуем логику игры. Нам нужен класс `Game24Env`, который:
1. Принимает строку с числами (например, "4 9 10 13").
2. Проверяет корректность предложенного решения (валидность выражения и равенство 24).
3. Генерирует новые задачи.

In [ ]:
class Game24Env:
    @staticmethod
    def validate_expression(numbers: str, expression: str) -> bool:
        """Проверяет, что выражение использует эти и только эти числа, и равно 24."""
        # 1. Проверка результата
        try:
            # Безопасный eval только для математики
            allowed = set("0123456789+-*/() ")
            if not set(expression).issubset(allowed):
                return False
            result = eval(expression)
            if abs(result - 24.0) > 0.001:
                return False
        except:
            return False
            
        # 2. Проверка использования чисел
        # Извлекаем все числа из выражения
        try:
            expr_nums = sorted(re.findall(r'\d+', expression))
            target_nums = sorted(numbers.split())
            return expr_nums == target_nums
        except:
            return False

print("Game24 environment defined.")

## Фаза 1: Baseline — Tree of Thoughts (ToT)

В качестве базы для сравнения возьмем упрощенную версию Tree of Thoughts. Она просто генерирует N вариантов решений и проверяет их, без умного поиска и бэкпропагации ошибок.

### Шаг 1.1: Реализация ToT

**Что мы будем делать:**
1. Создадим функцию `solve_tot`, которая просит LLM сгенерировать 5 разных выражений.
2. Проверим каждое через `Game24Env`.
3. Если найдено верное решение — возвращаем успех.

In [ ]:
async def solve_tot(numbers: str, n_thoughts: int = 5) -> str:
    console.print(f"--- [ToT] Solving for: {numbers} ---")
    
    prompt = f"""
    Use numbers {numbers} and basic arithmetic operations (+ - * /) to obtain 24.
    Each number must be used exactly once.
    Output only the mathematical expression, nothing else.
    Examples:
    Input: 4 4 6 8
    Output: (4 + 8) * (6 - 4)
    Input: 2 9 10 12
    Output: 2 * 12 * (10 - 9)
    """
    
    # Генерируем N вариантов параллельно или последовательно
    candidates = []
    for _ in range(n_thoughts):
        res = llm.invoke(prompt).content.strip()
        candidates.append(res)
    
    for expr in candidates:
        if Game24Env.validate_expression(numbers, expr):
            return f"SUCCESS: {expr}"
    
    return "FAILURE: No valid solution found"

print("Baseline ToT defined.")

## Фаза 2: Продвинутый подход — LATS с MCTS

Настоящая реализация LATS требует управления деревом поиска.

### Шаг 2.1: Класс узла дерева MCTS

**Что мы будем делать:**
Определим класс `MCTSNode`, который будет хранить:
- `state`: Текущее состояние (частичное или полное решение).
- `parent`: Родительский узел.
- `children`: Дочерние узлы.
- `visits`: Сколько раз мы посещали этот путь.
- `value`: Накопленная ценность (оценка качества).
- `reflection`: Текстовая самокритика/анализ этого шага.

Также реализуем метод `upper_confidence_bound` для выбора лучшего узла (баланс между эксплуатацией и разведкой).

In [ ]:
class MCTSNode:
    def __init__(self, state: str, parent=None, reflection: str = ""):
        self.state = state  # Например: "4 + 9 = 13. Left: 10, 13, 13"
        self.parent = parent
        self.children: List[MCTSNode] = []
        self.visits = 0
        self.value = 0.0
        self.reflection = reflection

    def uct_score(self, exploration_weight: float = 1.0) -> float:
        """Upper Confidence Bound for Trees (UCT)."""
        if self.visits == 0:
            return float('inf')
        
        # Средняя ценность (exploitation)
        q_value = self.value / self.visits
        # Фактор новизны (exploration)
        u_value = exploration_weight * math.sqrt(math.log(self.parent.visits) / self.visits)
        
        return q_value + u_value

    def add_child(self, child_state: str, reflection: str):
        child = MCTSNode(child_state, parent=self, reflection=reflection)
        self.children.append(child)
        return child
    
    def backpropagate(self, reward: float):
        """Обновление статистики узла и всех предков."""
        self.visits += 1
        self.value += reward
        if self.parent:
            self.parent.backpropagate(reward)
            
    def best_child(self):
        """Выбор дочернего узла с максимальным UCT."""
        if not self.children:
            return None
        return max(self.children, key=lambda c: c.uct_score())

### Шаг 2.2: Реализация LATS Агента

**Что мы будем делать:**
Реализуем класс `LATSAgent` с основным циклом:
1. **Select:** Начиная с корня, спускаемся по лучшим дочерним узлам (Select), пока не дойдем до листа.
2. **Expand:** Генерируем с помощью LLM несколько (k) возможных продолжений из этого узла.
3. **Evaluate/Reflect:** Для каждого продолжения просим LLM оценить его качество (0.0 - 1.0) и написать рефлексию.
4. **Backpropagate:** Поднимаем полученную оценку вверх по дереву.
5. **Repeat:** Повторяем цикл N раз или пока не найдем решение.

In [ ]:
class LATSAgent:
    def __init__(self, numbers: str, simulations: int = 5):
        self.root = MCTSNode(state=f"Start numbers: {numbers}")
        self.root.visits = 1  # Чтобы избежать log(0) в корне
        self.numbers = numbers
        self.simulations = simulations
        
    def solve(self) -> str:
        for i in range(self.simulations):
            console.print(f"\n[bold blue]--- Iteration {i+1}/{self.simulations} ---[/bold blue]")
            
            # 1. Selection
            node = self.select_node(self.root)
            console.print(f"Selected node: {node.state}")
            
            # Проверяем, не является ли состояние финальным решением
            if "= 24" in node.state or "equals 24" in node.state:
                return f"FOUND: {node.state}"
            
            # 2. Expansion
            candidates = self.expand_node(node)
            
            # 3. Evaluation & Backpropagation
            for cand_state in candidates:
                # Оценка и рефлексия
                reward, reflection = self.evaluate_and_reflect(cand_state)
                console.print(f"Candidate: [cyan]{cand_state}[/cyan] -> Reward: {reward:.2f}")
                # Создание нового узла
                child = node.add_child(cand_state, reflection)
                # Распространение награды
                child.backpropagate(reward)
                
                if reward == 1.0: # Нашли решение сразу
                    return f"FOUND: {cand_state}"
                    
        # После всех симуляций ищем лучший путь
        return self.get_best_solution()

    def select_node(self, node: MCTSNode) -> MCTSNode:
        """Жадный спуск по дереву на основе UCT."""
        current = node
        while current.children:
            current = current.best_child()
        return current

    def expand_node(self, node: MCTSNode, k: int = 3) -> List[str]:
        """Генерация K вариантов следующего шага."""
        prompt = f"""
        Current state: {node.state}
        Target: Make 24 using remaining numbers.
        
        Propose {k} diverse next possible steps. A step is ONE arithmetic operation between two numbers.
        Output just list of operations formatted as: 'op: <expression> = <result>, remaining: <nums>'
        Example: 'op: 4 + 8 = 12, remaining: 12, 6'
        """
        response = llm.invoke(prompt).content
        # Простой парсинг строк
        lines = [line.strip() for line in response.split('\n') if "op:" in line]
        return lines[:k] if lines else [response] # Fallback

    def evaluate_and_reflect(self, state: str) -> Tuple[float, str]:
        """LLM оценивает состояние (0.0-1.0) и пишет рефлексию."""
        prompt = f"""
        Evaluate this step towards solving Game of 24: "{state}"
        
        1. Is it a valid math operation?
        2. Are we getting closer to 24 or stuck?
        3. If it reached 24 exactly, score is 1.0
        
        Output JSON: {{"score": 0.0 to 1.0, "reflection": "analysis"}}
        """
        try:
            res = llm.invoke(prompt).content
            import json
            match = re.search(r'\{.*\}', res, re.DOTALL)
            if match:
                data = json.loads(match.group())
                return float(data.get("score", 0.0)), data.get("reflection", "")
        except:
            pass
        return 0.1, "Error evaluating"

    def get_best_solution(self) -> str:
        # Простой обход для поиска узла с макс value
        # В полноценной версии: путь от корня к лучшему листу
        return "Best attempt visualization placeholder"

print("LATS Agent class defined.")

## Фаза 3: Демонстрация и Сравнение

### Шаг 3.1: Тестирование на сложном примере

**Что мы будем делать:**
Запустим LATS на наборе чисел `4 9 10 13`, который считается достаточно сложным для простых агентов.

In [ ]:
tasks = ["4 9 10 13"]

# 1. Baseline ToT
print("Running Baseline ToT...")
# await solve_tot(tasks[0]) # Uncomment to run

# 2. LATS
console.print("\n[bold green]Running LATS Agent...[/bold green]")
agent = LATSAgent(tasks[0], simulations=6)
result = agent.solve()
console.print(Panel(result, title="LATS Result", border_style="green"))

## Заключение

LATS демонстрирует, как **поиск по дереву** в сочетании с **саморефлексией** позволяет агенту находить решения сложных задач, где линейное рассуждение часто заходит в тупик. Механизм Backpropagation позволяет "учить" агента избегать плохих путей в будущем в рамках той же сессии.